<a href="https://colab.research.google.com/github/Mario2712/DFL-Colab/blob/master/DFL_Colab_TagderoffenenT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to DFL-Colab!

Das ist eine adaptierte Version von DFL für den Tag der offenen Tür der FH Salzburg.

Adaptiert und getestet von René Pichler und Mario Siller

Version 1.0



# Check GPU

*   Google Colab can provide you with one of two Tesla graphics cards: K80, T4, P4 or P100
*   Here you can check the model of GPU before using DeepFaceLab



In [0]:
!nvidia-smi

# Clone Github repository and install requirements

* Clone Github repository or pull updates
* Requirements install is automatically

In [0]:
#@title Clone or pull DeepFaceLab from Github

Mode = "clone" #@param ["clone", "pull"]

from pathlib import Path
if (Mode == "clone"):
  !git clone https://github.com/Mario2712/DeepFaceLab.git
else:
  %cd /content/DeepFaceLab
  !git pull

!pip install -r /content/DeepFaceLab/requirements-colab.txt
!pip install --upgrade scikit-image
!apt-get install cuda-10-0
!pip install torch===1.3.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchvision===0.4.0 -f https://download.pytorch.org/whl/torch_stable.html

if not Path("/content/workspace").exists():
  !wget -q --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=1hTH2h6l_4kKrczA8EkN6GyuXx4lzmCnK' -O pretrain_CelebA.zip
  !mkdir /content/pretrain
  !unzip -q /content/pretrain_CelebA.zip -d /content/pretrain/
  !rm /content/pretrain_CelebA.zip

print("Done!")

#Manage workspace



*   You can import/export workspace or individual data, like model files with Google Drive
*   Also, you can use HFS (HTTP Fileserver) for directly import/export you workspace from your computer
*   You can clear all workspace or delete part of it



In [0]:
#@title Import from Drive

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
  
print("Done!")



In [0]:
#@title Delete and recreate
Mode = "Delete and recreate workspace" #@param ["Delete and recreate workspace", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content" 

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/workspace ; mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"  
elif Mode == "Delete data_dst":
  cmd = "rm /content/workspace/data_dst/*.png || rm /content/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/workspace/data_dst/merged"
  
!$cmd
print("Done!")

# Extract, sorting and faceset tools
* Extract frames for SRC or DST video.
* Denoise SRC or DST video. "Factor" param set intesity of denoising
* Detect and align faces with one of detectors. (S3FD - recommended). If you need, you can get frames with debug landmarks.
* Options "S3FD (Mark only)" and "S3FD (Extract unaligned)" are needed for the AVATAR model. Before using, read the guide in the DeepFaceLab repository.
* Export workspace to Google Drive after extract and sort it manually (In "Manage Workspace" block)
* You can enhance your facesets with DFL FacesetEnhancer. Enhanced images will replace yours in "aligned" directory.


In [0]:
#@title Extract frames
Video = "data_dst" #@param ["data_dst"]

%cd "/content"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/ --fps 10 --output-ext png"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/ --fps 0 --output-ext png"
  
!python $cmd

In [0]:
#@title Detect faces
Data = "data_dst" #@param ["data_dst"]
Detector = "S3FD" #@param ["S3FD"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
facetype = None

if Detector == "S3FD":
  detect_type = "s3fd"
elif Detector == "S3FD (Mark only)":
  detect_type = "s3fd"
  facetype = "mark_only"
elif Detector == "S3FD (Extract unaligned)":
  detect_type = "s3fd"
  facetype = "full_face_no_align"
elif Detector == "MT":
  detect_type = "mt"

folder = "workspace/"+Data
folder_align = folder+"/aligned"
debug_folder = folder_align+"/debug"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_align

if Debug:
  cmd+= " --debug-dir "+debug_folder

cmd+=" --detector "+detect_type
  
if facetype:
  cmd+=" --face-type "+facetype
  
%cd "/content"
!python $cmd

In [0]:
#@title Sort aligned
Data = "data_dst" #@param ["data_dst"]
sort_type = "absdiff" #@param ["absdiff"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

# Train model

* Choose your model type, but SAEHD is recommend for everyone
* Set model options on output field
* You can see preview manually, if go to model folder in filemanager and double click on preview.jpg file
* Your workspace will be archived and upload to mounted Drive after 11 hours from start session
* If you select "Backup_every_hour" option, your workspace will be backed up every hour.
* Also, you can export your workspace manually in "Manage workspace" block

In [0]:
#@title Training
Model = "SAEHD" #@param ["SAEHD"]
Backup_every_hour = False #@param {type:"boolean"}

%cd "/content"

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if (Backup_every_hour):
  if not os.path.exists('workspace.zip'):
    print("Creating workspace archive ...")
    !zip -r -q workspace.zip workspace
    print("Archive created!")
  else:
    print("Archive exist!")

if (Backup_every_hour):
  print("Time to end session: "+str(round((43200-uptime)/3600))+" hours")
  backup_time = str(3600)
  backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace/model'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
elif (round(39600-uptime) > 0):
  print("Time to backup: "+str(round((39600-uptime)/3600))+" hours")
  backup_time = str(round(39600-uptime))
  backup_cmd = " --execute-program "+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""
    
cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain/aligned --model-dir workspace/model --model "+Model
  
if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

!python $train_cmd

# Convert frames

In [0]:
#@title Convert
Model = "SAEHD" #@param ["SAEHD"]

if Model == "AVATAR":  
  cmd = "DeepFaceLab/main.py convert --input-dir workspace/data_dst/aligned --output-dir workspace/data_dst/merged --model-dir workspace/model --model "+Model
else:
  cmd = "DeepFaceLab/main.py convert --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [0]:
#@title Get result video and copy to Drive 

!python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --fps 25 --bitrate 16 
!cp /content/workspace/result.mp4 /content/drive/My\ Drive/